# Actividad en clase 5: Simulación del VaR por Montecarlo 
#### Luis Eduardo Jiménez del Muro
#### 26/02/2025
---

In [2]:
import numpy as np
import yfinance as yf
import pandas as pd
import random
import itertools

# Tickers del S&P500

In [4]:
sp = pd.read_excel('sp_500.xlsx')[['Symbol', 'GICS_Sector']]
sp.head()

,Symbol,GICS_Sector
0,MMM,Industrials
1,AOS,Industrials
2,ABT,Health Care
3,ABBV,Health Care
4,ACN,Information Technology


In [5]:
closes_sp = yf.download(sp['Symbol'].tolist(), start='2022-01-01', end='2025-01-01')['Close']
closes_sp

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  503 of 503 completed

16 Failed downloads:
['MRO', 'WRK', 'CTLT', 'FLT', 'PXD']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2025-01-01) (Yahoo error = "No data found, symbol may be delisted")')
['BRK.B', 'DISH', 'PKI', 'PEAK', 'CDAY', 'FISV', 'ATVI', 'ABC', 'RE', 'FRC']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2025-01-01)')


Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,153.272293,18.750000,218.612701,178.879898,119.733452,NaN,130.754242,42.362530,389.190277,564.369995,...,85.903877,61.547108,56.641762,53.827461,112.315758,128.309433,122.322243,583.900024,56.988926,227.280457
2022-01-04,148.090714,19.020000,218.862000,176.609650,119.503563,NaN,127.679092,42.914051,386.408997,554.000000,...,85.629845,61.936649,58.772297,54.101852,113.507225,128.995483,123.563194,587.599976,58.612377,218.627426
2022-01-05,145.553818,18.680000,218.308060,171.911835,120.131317,NaN,127.105438,42.410072,379.604095,514.429993,...,82.458725,62.543610,59.503262,53.231377,111.960236,127.360283,123.155853,558.179993,58.745449,210.314301
2022-01-06,146.063171,18.570000,223.109024,169.042084,119.565468,NaN,127.086647,42.657307,361.272888,514.119995,...,82.840431,62.054417,60.902817,53.193527,111.806496,128.675980,122.132774,555.159973,61.158424,211.149475
2022-01-07,142.174500,19.280001,219.822205,169.209137,119.256012,NaN,127.481621,42.856995,354.343689,510.700012,...,82.752350,62.597939,61.402012,53.363842,110.912903,127.200493,121.393898,530.859985,62.178616,205.002090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,135.848907,17.350000,43.489883,257.916443,178.318420,NaN,114.160919,92.669998,361.630005,447.940002,...,88.535454,67.649513,105.456718,18.742170,117.784721,134.365067,107.059036,395.440002,54.543877,164.203583
2024-12-26,135.579407,17.350000,43.997086,258.735504,177.525894,NaN,114.668251,92.930000,360.429993,450.160004,...,89.063995,67.619759,105.545921,19.000000,118.103729,135.360367,107.009148,396.850006,54.920929,165.021118
2024-12-27,135.289932,17.350000,43.818073,255.309296,176.347015,NaN,114.389717,92.339996,356.179993,446.480011,...,88.435730,67.580086,105.536011,18.940001,117.166634,134.593979,106.630005,389.070007,54.256123,164.103897


In [9]:
closes_sp = closes_sp.dropna(axis=1)
closes_sp.head()

Ticker,A,AAL,AAP,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,153.272293,18.750000,218.612701,178.879898,119.733452,130.754242,42.362530,389.190277,564.369995,167.763397,...,85.903877,61.547108,56.641762,53.827461,112.315758,128.309433,122.322243,583.900024,56.988926,227.280457
2022-01-04,148.090714,19.020000,218.862000,176.609650,119.503563,127.679092,42.914051,386.408997,554.000000,166.248169,...,85.629845,61.936649,58.772297,54.101852,113.507225,128.995483,123.563194,587.599976,58.612377,218.627426
2022-01-05,145.553818,18.680000,218.308060,171.911835,120.131317,127.105438,42.410072,379.604095,514.429993,163.681763,...,82.458725,62.543610,59.503262,53.231377,111.960236,127.360283,123.155853,558.179993,58.745449,210.314301
2022-01-06,146.063171,18.570000,223.109024,169.042084,119.565468,127.086647,42.657307,361.272888,514.119995,164.202621,...,82.840431,62.054417,60.902817,53.193527,111.806496,128.675980,122.132774,555.159973,61.158424,211.149475
2022-01-07,142.174500,19.280001,219.822205,169.209137,119.256012,127.481621,42.856995,354.343689,510.700012,159.893692,...,82.752350,62.597939,61.402012,53.363842,110.912903,127.200493,121.393898,530.859985,62.178616,205.002090


## Filtrar los tickers que si se descargaron

In [21]:
tickers_sp = closes_sp.columns.to_list()
tickers_sector = sp[sp["Symbol"].isin(tickers_sp)]
tickers_sector.head()

,Symbol,GICS_Sector
0,MMM,Industrials
1,AOS,Industrials
2,ABT,Health Care
3,ABBV,Health Care
4,ACN,Information Technology


In [29]:
def sample_stocks(df, sample_size=8, max_per_sector=2):
    sampled = []
    sector_counts = {}

    shuffled_df = df.sample(frac=1).reset_index(drop=True) 

    for _, row in shuffled_df.iterrows():
        sector = row["GICS_Sector"]
        if sector_counts.get(sector, 0) < max_per_sector:
            sampled.append(row)
            sector_counts[sector] = sector_counts.get(sector, 0) + 1
        if len(sampled) == sample_size:
            break

    return pd.DataFrame(sampled)

samples = [sample_stocks(tickers_sector) for _ in range(50)]
samples[0]


,Symbol,GICS_Sector
0,IPG,Communication Services
1,BBWI,Consumer Discretionary
2,BR,Information Technology
3,INTU,Information Technology
4,MLM,Materials
5,ZBH,Health Care
8,VRTX,Health Care
9,GE,Industrials


In [30]:
tickers_port = []
for i in samples:
    tickers_port.append(i['Symbol'].tolist())

In [31]:
n_tickers_selected = 8
n_combinations = 50

In [34]:
list_prices_ports = []
for i in tickers_port:
    list_prices_ports.append(closes_sp[i])

In [45]:
import numpy as np
import pandas as pd

def simulate_portfolios(list_prices_ports, tickers_port, num_portafolios=100):
    """
    Simula diferentes portafolios generando combinaciones de pesos aleatorios y calculando su 
    rendimiento y riesgo esperado.

    Parámetros:
    - list_prices_ports: lista de DataFrames, cada uno con los precios de cierre de un activo.
    - tickers_port: lista de listas, donde cada sublista contiene tickers de una combinación de activos.
    - num_portafolios: número de simulaciones a realizar (por defecto 10,000).

    Retorna:
    - all_portfolios: lista de DataFrames, cada uno con los resultados de una simulación de portafolio.
    """
    all_portfolios = []

    # Crear un diccionario para acceder rápidamente a los DataFrames por su ticker
    price_dict = {df.columns[0]: df for df in list_prices_ports}

    for combination in tickers_port:
        # Filtrar los DataFrames en función de los tickers en la combinación
        selected_data = [price_dict[ticker] for ticker in combination if ticker in price_dict]

        if len(selected_data) != len(combination):
            print(f"Advertencia: Algunos tickers en {combination} no fueron encontrados en los datos.")
            continue

        # Unir los precios en un solo DataFrame
        closes_sp = pd.concat(selected_data, axis=1)

        # Calcular los rendimientos diarios
        returns = closes_sp.pct_change().dropna()

        # Matriz de covarianza de los retornos
        covariance_matrix = returns.cov()

        # Calcular retornos y covarianza anualizados
        annual_returns = returns.mean() * 252
        annual_cov = covariance_matrix * 252

        # Inicializar arrays para los resultados
        n_tickers_selected = len(combination)
        weights_array = np.zeros((num_portafolios, n_tickers_selected))
        returns_array = np.zeros(num_portafolios)
        risk_array = np.zeros(num_portafolios)

        for i in range(num_portafolios):
            # Generar pesos aleatorios para cada portafolio
            weights = np.random.random(n_tickers_selected)
            weights /= np.sum(weights)

            # Calcular rendimiento y volatilidad esperada anual
            portfolio_return = np.dot(annual_returns, weights)
            portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))

            # Guardar los valores en los arrays
            weights_array[i, :] = weights
            returns_array[i] = portfolio_return
            risk_array[i] = portfolio_risk

        # Crear DataFrame con los resultados del portafolio
        portfolios = pd.DataFrame({
            "Return": returns_array,
            "Risk": risk_array
        })

        # Agregar las columnas de pesos para cada activo
        for i, symbol in enumerate(combination):
            portfolios[symbol + " Weight"] = weights_array[:, i]

        # Guardar el resultado
        all_portfolios.append(portfolios)

    return all_portfolios
